In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from datetime import datetime
import re
import warnings
import os
from textblob import TextBlob
import talib
from matplotlib.gridspec import GridSpec
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
DATA_DIR = "/notebooks/data"
NEWS_PATH = os.path.join(DATA_DIR, "raw_analyst_ratings.csv")
TICKERS = ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA']
INITIAL_CAPITAL = 10000

In [9]:
print("=" * 80)
print("📥 LOADING LOCAL DATA")
print("=" * 80)

📥 LOADING LOCAL DATA


In [ ]:
import os
df_news = None
if os.path.exists(NEWS_PATH):
    df = pd.read_csv(NEWS_PATH)
    print(f"✅ News loaded: {len(df)} records")

    # Handle date
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    if df['date'].dt.tz is None:
        df['date'] = df['date'].dt.tz_localize('UTC', ambiguous='infer')
    else:
        df['date'] = df['date'].dt.tz_convert('UTC')
    df = df.dropna(subset=['date'])
    df['date_only'] = df['date'].dt.date
    print(f"   Date range: {df['date'].min()} to {df['date'].max()}")

    # Store
    df_news = df
else:
    print("⚠️ No news file found at:", NEWS_PATH)
    exit()

⚠️ No news file found at: /notebooks/data\raw_analyst_ratings.csv


: 

In [2]:
import os
df_news = None
if os.path.exists(NEWS_PATH):
    df = pd.read_csv(NEWS_PATH)
    print(f"✅ News loaded: {len(df)} records")

    # Handle date
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    if df['date'].dt.tz is None:
        df['date'] = df['date'].dt.tz_localize('UTC', ambiguous='infer')
    else:
        df['date'] = df['date'].dt.tz_convert('UTC')
    df = df.dropna(subset=['date'])
    df['date_only'] = df['date'].dt.date
    print(f"   Date range: {df['date'].min()} to {df['date'].max()}")

    # Store
    df_news = df
else:
    print("⚠️ No news file found at:", NEWS_PATH)
    exit()

NameError: name 'NEWS_PATH' is not defined